In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
# df = spark.table("field_ai_examples.alphaleger.sec_rag_docs_pages_bronze")
df = spark.table("field_ai_examples.alphaleger.sec_rag_docs_pages_platinum")
# display(df)
# display(df.select("company").distinct())
df.filter(df.company.isin(["apple", "american express"])).repartition(1).write.mode("overwrite").parquet("/Volumes/felixflory/databricks_genai_hackathon/raw_data/sec_rag_docs_pages_folder.parquet")

In [0]:
import sys, os
sys.path.append(os.path.abspath('..'))
from configs.project import projectConfig

In [0]:
import urllib.request

url = "https://raw.githubusercontent.com/fflory/databricks_genai_hackathon/main/data/sec_rag_docs_pages.snappy.parquet"
local_path = "/Volumes/felixflory/databricks_genai_hackathon/raw_data/sec_rag_docs_pages.snappy.parquet"
_ = urllib.request.urlretrieve(url, local_path)

In [0]:
spark.read.parquet(
    f"/Volumes/{projectConfig.uc_catalog}/{projectConfig.uc_schema}/raw_data/sec_rag_docs_pages.snappy.parquet"
).write.format("delta").mode("overwrite").saveAsTable(
    projectConfig.table_sec_rag_docs_pages, mode="overwrite",
)

In [0]:
catalog = projectConfig.uc_catalog
schema = projectConfig.uc_schema

In [0]:
# spark.sql(f"ALTER TABLE {projectConfig.table_sec_rag_docs_pages} ALTER COLUMN <PK_COLUMN> SET NOT NULL")

# spark.sql(f"ALTER TABLE {projectConfig.table_sec_rag_docs_pages} ADD CONSTRAINT {sec_rag_docs_pages}_pk PRIMARY KEY( <PK_COLUMN> )")

spark.sql(f"ALTER TABLE {projectConfig.table_sec_rag_docs_pages} SET TBLPROPERTIES (delta.enableChangeDataFeed = true) ")
